In [11]:
import os
import pandas as pd
from networkx.algorithms.bipartite.basic import color


def get_dataset(dataset_path):
    if not os.path.exists(dataset_path):
        print("Dataset not found")
        return None
    dataset = []
    
    for folder in os.listdir(dataset_path):
        if not os.path.isdir(os.path.join(dataset_path, folder)):
            continue
        for file in os.listdir(dataset_path + '/' + folder):
            if not file.endswith('.png'):
                continue
            csv_file = dataset_path + '/' + folder + '/' + file[:-4] + '.csv'
            res_df = pd.read_csv(csv_file)
            image = os.path.join(dataset_path, folder, file)
            y = res_df['y'].to_numpy()
            dataset.append([image, y])
            
    return pd.DataFrame(dataset, columns=['image', 'y'])
                
dataset = get_dataset('synthetic_dataset')
dataset

,image,y
0,synthetic_dataset/135/c01-014.png,"[900, 1100, 1250, 1600, 1800, 1950, 2150, 2300..."
1,synthetic_dataset/135/c01-009.png,"[900, 1050, 1300, 1450, 1600, 1800, 1850, 2000..."
2,synthetic_dataset/307/g02-073.png,"[750, 900, 1150, 1250, 1450]"
3,synthetic_dataset/551/n06-194.png,"[700, 900, 1100, 1250, 1400, 1600, 1800, 1950]"
4,synthetic_dataset/551/n06-156.png,"[700, 1300, 1450, 1650, 2000]"
5,synthetic_dataset/551/n06-182.png,"[750, 1100, 1300, 1450, 1600, 1800, 1950]"
6,synthetic_dataset/551/n06-169.png,"[700, 900, 1100, 1300, 1450, 1600, 1800]"
7,synthetic_dataset/551/n06-186.png,"[750, 850, 1050, 1300, 1400, 1650, 1800, 1950]"
8,synthetic_dataset/551/n06-163.png,"[750, 850, 1050, 1250, 1450, 1650, 1800, 2200,..."
9,synthetic_dataset/551/n06-201.png,"[700, 950, 1100, 1250, 1450, 1650, 1800, 1950]"


In [18]:
from PIL import Image, ImageDraw

def get_bounding_boxes(image_path, y, height=150):
    image = Image.open(image_path)
    boxes = []
    for y_i in y:
        x1, x2 = 0, image.width
        y1, y2 = y_i, y_i + height
        boxes.append([x1, y1, x2, y2])
    return boxes

def draw_bounding_boxes(image, boxes):
    # convert to RGB
    image = image.convert('RGB')
    draw = ImageDraw.Draw(image)
    colors = ['red', 'blue']
    for index, box in enumerate(boxes):
        box = [box[0] + 10, box[1] + 10, box[2] - 10, box[3] - 10]
        draw.rectangle(box, outline=colors[index % 2], width=1)
    return image

boxes = get_bounding_boxes(dataset.iloc[0]['image'], dataset.iloc[0]['y'])
image = Image.open(dataset.iloc[0]['image'])
draw_bounding_boxes(image, boxes).show()

In [ ]:
import numpy as np
import os
from PIL import Image, ImageDraw

image = cppe5["train"][2]["image"]
annotations = cppe5["train"][2]["objects"]
draw = ImageDraw.Draw(image)

categories = cppe5["train"].features["objects"].feature["category"].names

id2label = {index: x for index, x in enumerate(categories, start=0)}
label2id = {v: k for k, v in id2label.items()}

for i in range(len(annotations["id"])):
    box = annotations["bbox"][i]
    class_idx = annotations["category"][i]
    x, y, w, h = tuple(box)
    # Check if coordinates are normalized or not
    if max(box) > 1.0:
        # Coordinates are un-normalized, no need to re-scale them
        x1, y1 = int(x), int(y)
        x2, y2 = int(x + w), int(y + h)
    else:
        # Coordinates are normalized, re-scale them
        x1 = int(x * width)
        y1 = int(y * height)
        x2 = int((x + w) * width)
        y2 = int((y + h) * height)
    draw.rectangle((x, y, x + w, y + h), outline="red", width=1)
    draw.text((x, y), id2label[class_idx], fill="white")

image